In [82]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import alpaca_trade_api as tradeapi
import datetime as dt
import pytz

_timeNow = dt.datetime.now(pytz.timezone('US/Eastern'))
_1YearsAgo = _timeNow - dt.timedelta(days=360)
_2YearsAgo = _timeNow - dt.timedelta(days=730)
_3YearsAgo = _timeNow - dt.timedelta(days=1095)

ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

# Create a connection to the API 

api = tradeapi.REST("PKBIB56DKVHDGM34ZVOP", "l3Gqb5qW6qst8IgtZiXxIVr4LSFws9njYTkFAD6e", ALPACA_API_BASE_URL, api_version="v2")

# Set the ticker symbol and the number of shares to buy
ticker = "BTCUSD"


In [83]:
# Make API call
ETH_prices = api.get_crypto_bars(ticker, '1day',
                         start=_3YearsAgo.isoformat(),
                         end=None,
                         limit=None).df
#Filter data by exchange
ETH_prices = ETH_prices[ETH_prices.exchange == 'CBSE']
# Setting these options will allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)
# Create a signals_df DataFrame that is a copy of the ixn_df Dataframe
signals_df = ETH_prices.copy()
signals_df.drop(columns=['exchange','volume','trade_count','vwap'],inplace=True)
signals_df.tail()

,close,high,low,open
timestamp,,,,
2022-10-23 05:00:00+00:00,19337.61,19698.10,19070.92,19188.65
2022-10-24 05:00:00+00:00,19341.22,19453.35,19159.35,19337.10
2022-10-25 05:00:00+00:00,20211.45,20420.88,19240.76,19341.22
2022-10-26 05:00:00+00:00,20806.59,21022.81,20136.82,20211.46
2022-10-27 05:00:00+00:00,20300.21,20878.17,20160.72,20806.59


In [84]:
from finta import TA
# Set the short window and long windows
short_window = 15
long_window = 50

# Add the SMA technical indicators for the short and long windows
signals_df["short_EMA"] = TA.DEMA(signals_df, short_window)
signals_df["long_EMA"] = TA.DEMA(signals_df, long_window)

In [85]:
# Visualize close price for the investment
security_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['short_EMA', 'long_EMA']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)
dema_graph = security_close * moving_avgs
dema_graph.opts(
    title="BTC Algorithm - Double exponential moving average over price",
    yformatter='%.0f'
)

:Overlay
   .Curve.I     :Curve   [timestamp]   (close)
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)

In [86]:
# Prepopulate the `Signal` for trading
signals_df['Signal'] = 0.0
signals_df = signals_df.dropna()
signals_df


# Generate the trading signal 0 or 1,
# where 1 is when short-window (SMA50) is greater than the long (SMA 100)
# and 0 otherwise
signals_df['Signal'][short_window:] = np.where(
    signals_df['short_EMA'][short_window:]> signals_df['long_EMA'][short_window:], 1.0, 0.0 )

##### Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Review the DataFrame
# Review the DataFrame
signals_df=signals_df.dropna()

# Set initial capital
initial_capital = float(1000000)

# Set the share size
share_size = 3


# Buy a 500 share position when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)
signals_df['Position'] = share_size * signals_df['Signal']

# Determine the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply the close price by the number of shares held, or the Position
signals_df['Portfolio Holdings'] = signals_df['close'] * signals_df['Position']

# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
signals_df['Portfolio Cash'] = initial_capital - (signals_df['close'] * signals_df['Entry/Exit Position']).cumsum() 

# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

signals_df.dropna()
signals_df.loc["2020-02-20":"2020-02-28",:]

,close,high,low,open,short_EMA,long_EMA,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
timestamp,,,,,,,,,,,,,
2020-02-20 06:00:00+00:00,9692.70,9747.78,9578.14,9610.05,10016.935740,9907.842993,1.0,0.0,3.0,0.0,29078.10,976630.42,1005708.52
2020-02-21 06:00:00+00:00,9695.66,9769.66,9567.56,9689.77,9963.856339,9920.245420,1.0,0.0,3.0,0.0,29086.98,976630.42,1005717.40
2020-02-22 06:00:00+00:00,9900.01,9949.00,9667.13,9668.23,9966.921395,9947.186298,1.0,0.0,3.0,0.0,29700.03,976630.42,1006330.45
2020-02-23 06:00:00+00:00,9772.63,10030.00,9585.00,9965.00,9938.367896,9962.073028,0.0,-1.0,0.0,-3.0,0.00,1005948.31,1005948.31
2020-02-24 06:00:00+00:00,9554.77,9669.99,9532.51,9660.00,9862.856026,9958.610211,0.0,0.0,0.0,0.0,0.00,1005948.31,1005948.31
2020-02-25 06:00:00+00:00,9188.85,9375.36,9092.92,9304.99,9714.465899,9926.420060,0.0,0.0,0.0,0.0,0.00,1005948.31,1005948.31
2020-02-26 06:00:00+00:00,8778.30,9280.00,8609.91,9188.84,9496.419299,9863.767063,0.0,0.0,0.0,0.0,0.00,1005948.31,1005948.31
2020-02-27 06:00:00+00:00,8786.54,8899.94,8779.06,8814.90,9320.188010,9804.929984,0.0,0.0,0.0,0.0,0.00,1005948.31,1005948.31
2020-02-28 06:00:00+00:00,8708.89,8813.12,8428.80,8786.53,9158.723463,9743.063596,0.0,0.0,0.0,0.0,0.00,1005948.31,1005948.31


In [87]:
signals_df.tail()

,close,high,low,open,short_EMA,long_EMA,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
timestamp,,,,,,,,,,,,,
2022-10-23 05:00:00+00:00,19337.61,19698.10,19070.92,19188.65,19186.917962,18780.143557,1.0,0.0,3.0,0.0,58012.83,983813.17,1041826.00
2022-10-24 05:00:00+00:00,19341.22,19453.35,19159.35,19337.10,19211.417252,18784.652042,1.0,0.0,3.0,0.0,58023.66,983813.17,1041836.83
2022-10-25 05:00:00+00:00,20211.45,20420.88,19240.76,19341.22,19436.162845,18856.584816,1.0,0.0,3.0,0.0,60634.35,983813.17,1044447.52
2022-10-26 05:00:00+00:00,20806.59,21022.81,20136.82,20211.46,19759.833489,18970.832433,1.0,0.0,3.0,0.0,62419.77,983813.17,1046232.94
2022-10-27 05:00:00+00:00,20300.21,20878.17,20160.72,20806.59,19905.316584,19040.180974,1.0,0.0,3.0,0.0,60900.63,983813.17,1044713.80


In [88]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)
# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="BTC Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [timestamp]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [timestamp]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [timestamp]   (Portfolio Total)

In [89]:
limit_amount = round(signals_df['close'][-1],0)
limit_amount

20300.0

In [90]:
Signal = int(signals_df['Entry/Exit Position'][-1])
Signal

0

In [91]:
# build if else statement to submit the order
if Signal > 0:
    orderSide = 'buy'
elif Signal < 0:
    orderSide = 'sell'
elif Signal == 0:
    submit = False 
    print("No order is taken place")

if submit is not False:
    api.submit_order(
        symbol = ticker,
        qty = number_of_shares,
        side = orderSide,
        time_in_force = 'gtc',
        type= 'limit',
        limit_price = limit_amount
    )

No order is taken place
